In [2]:
import openvsp as vsp
from import_prop import process_propeller_file 
import os
import numpy as np
import matplotlib.pyplot as plt

stdout = vsp.cvar.cstdout
errorMgr = vsp.ErrorMgrSingleton.getInstance()

In [18]:
geometry_name = "10x10E-PERF"

In [55]:
process_propeller_file('./geometry/' + geometry_name + '.PE0')

After importing BEM file, set the following properties:
Construction X/C: 0.000
    Feather Axis: 0.125


'./geometry/10x10E-PERF.bem'

In [56]:
os.remove(geometry_name + ".vsp3")

In [61]:
# Set up the geometry
vsp.VSPCheckSetup()
vsp.VSPRenew()
vsp.ClearVSPModel()
vsp.DeleteAllResults()


# Import the propeller geometry
prop_id = vsp.ImportFile('geometry/' + geometry_name + '.bem', vsp.IMPORT_BEM, "")

vsp.SetParmVal(prop_id, "ConstructXoC", "Design", 0.0)  # Construct x/c at 0.0
vsp.SetParmVal(prop_id, "FeatherAxisXoC", "Design", 0.125)  # Set the feather axis to 0.125


# Set up root airfoil section
xsec_surf = vsp.GetXSecSurf(prop_id, 0)
vsp.ChangeXSecShape(xsec_surf, 0, vsp.XS_FILE_AIRFOIL)

xsec = vsp.GetXSec(xsec_surf, 0)
vsp.ReadFileAirfoil(xsec, "./geometry/e63.dat")
vsp.InsertXSec(prop_id, 0, vsp.XS_FILE_AIRFOIL)

# Set up mid airfoil section
xsec = vsp.GetXSec(xsec_surf, 1)
vsp.SetParmVal(vsp.GetXSecParm(xsec, "RadiusFrac"),  0.732)
vsp.ReadFileAirfoil(xsec, "./geometry/n4412.dat")

# Set up tip airfoil section
vsp.ChangeXSecShape(xsec_surf, 2, vsp.XS_FILE_AIRFOIL)

xsec = vsp.GetXSec(xsec_surf, 2)
vsp.ReadFileAirfoil(xsec, "./geometry/n4412.dat")

# Set tesselation
vsp.SetParmVal(prop_id, "Tess_U", "Shape", 50)  # Num_U 
vsp.SetParmVal(prop_id, "Tess_W", "Shape", 33)  # Num_W

vsp.SetParmVal(prop_id, "OutCluster", "Design", 0.2) # Set clustering 
vsp.SetParmVal(prop_id, "LECluster", "Design", 0.2)
vsp.SetParmVal(prop_id, "TECluster", "Design", 0.2) 
vsp.SetParmVal(prop_id, "CapUMaxOption", "EndCap", 4) # Set end cap to sharp 

# Set up simulation parameters for easy repeatability.
vsp.SetParmVal(vsp.FindParm(vsp.FindUnsteadyGroup(0),"RPM","UnsteadyGroup"), 5000)

aero_id = vsp.FindContainer( "VSPAEROSettings", 0)
vsp.SetParmVal(aero_id, "Rho", "VSPAERO", 1.225) 
vsp.SetParmVal(aero_id, "Vinf", "VSPAERO", 12) 
vsp.SetParmVal(aero_id, "RotateBladesFlag", "VSPAERO", 1)
vsp.SetParmVal(aero_id, "AutoTimeNumRevs", "VSPAERO", 2)
vsp.SetParmVal(aero_id, "RootWakeNodes", "VSPAERO", 64)
vsp.SetParmVal(aero_id, "NCPU", "VSPAERO", 14)

vsp.Update()
vsp.WriteVSPFile(geometry_name + ".vsp3")

In [62]:
os.startfile(geometry_name + ".vsp3")

In [15]:
vsp.SetParmVal(vsp.FindParm(vsp.FindUnsteadyGroup(0),"RPM","UnsteadyGroup"), 5000)

us_group = vsp.GetUnsteadyGroupName(0)
# parm_container_id = vsp.FindContainer(us_group, 0)
parm_container_id = vsp.FindUnsteadyGroup(0)
parm_ids = vsp.FindContainerParmIDs( parm_container_id )

for i in range(len(parm_ids)):

    name_id = vsp.GetParmName( parm_ids[i] ) + ": " + str(vsp.GetParmVal(parm_ids[i])) + "\n"

    print( name_id )

GeomPropertyType: 2.0

RotorDia: 0.254

Ox: 0.0

Oy: 0.0

Oz: 0.0

Rx: -1.0

Ry: 0.0

Rz: 0.0

Vx: 0.0

Vy: 0.0

Vz: 0.0

Ax: 0.0

Ay: 0.0

Az: 0.0

RPM: 5000.0

Mass: 0.0

Ixx: 0.0

Iyy: 0.0

Izz: 0.0

Ixy: 0.0

Ixz: 0.0

Iyz: 0.0



In [14]:
vsp.SetAnalysisInputDefaults("VSPAEROSweep")

In [9]:
#VSPAEROSinglePoint
c
vsp.WriteVSPFile("apc_bem_10x10.vsp3")

In [27]:
parm_array = vsp.GetGeomParmIDs(prop_id)

for i in parm_array:
    name = vsp.GetParmName(i)
    # name = vsp.GetParmDisplayGroupName(i)
    print(name)

X_Location
Y_Location
Z_Location
X_Rotation
Y_Rotation
Z_Rotation
Origin
X_Rel_Location
Y_Rel_Location
Z_Rel_Location
X_Rel_Rotation
Y_Rel_Rotation
Z_Rel_Rotation
Abs_Or_Relitive_flag
Trans_Attach_Flag
Rots_Attach_Flag
U_Attach_Location
U_01
U_Attach_Location0N
V_Attach_Location
R_Attach_Location
R_01
R_Attach_Location0N
S_Attach_Location
T_Attach_Location
L_Attach_Location
L_01
L_Attach_Location0Len
M_Attach_Location
N_Attach_Location
Eta_Attach_Location
Scale
Last_Scale
Tess_U
Tess_W
Wake
WakeScale
WakeAngle
X_Len
Y_Len
Z_Len
X_Min
Y_Min
Z_Min
Sym_Ancestor
Sym_Ancestor_Origin_Flag
Sym_Planar_Flag
Sym_Axial_Flag
Sym_Rot_N
Density
Mass_Area
Mass_Prior
Shell_Flag
PointMass
CGx
CGy
CGz
Ixx
Iyy
Izz
Ixy
Ixz
Iyz
Negative_Volume_Flag
CapUMinOption
CapUMinLength
CapUMinOffset
CapUMinStrength
CapUMinSweepFlag
CapUMinTess
CapUMaxOption
CapUMaxLength
CapUMaxOffset
CapUMaxStrength
CapUMaxSweepFlag
PercLam
FFBodyEqnType
FFWingEqnType
FFUser
Q
Roughness
TeTwRatio
TawTwRatio
IncorporatedGen
Expanded

In [ ]:
# Set up the geometry
vsp.ClearVSPModel()

prop_id = vsp.AddGeom("PROP", "")
errorMgr.PopErrorAndPrint(stdout)

# Set number of blades and diameter
vsp.SetParmValUpdate(prop_id, "PropMode", "Design", vsp.PROP_BLADES)
vsp.SetParmVal(prop_id, "NumBlade", "Design", 2)  # Number of blades
vsp.SetParmVal(prop_id, "Diameter", "Design", 0.254)  # Diameter in meters


# Set up the airfoil sections
xsec_surf = vsp.GetXSecSurf(prop_id, 0)
vsp.ChangeXSecShape(xsec_surf, 1, vsp.XS_FILE_AIRFOIL)

xsec = vsp.GetXSec(xsec_surf, 1)
vsp.ReadFileAirfoil(xsec, "./geometry/e63.dat")
errorMgr.PopErrorAndPrint(stdout)


# vsp.SetParmVal(prop_id, "BladePitch", "XSec_1", 30.0)  # Example for pitch in degrees
errorMgr.PopErrorAndPrint(stdout)

vsp.Update()
vsp.WriteVSPFile("apc_10x10.vsp3")

# Set up the analysis
analysis_name = "VSPAEROPropeller"
vsp.SetAnalysisInputDefaults(analysis_name)

